In [1]:
# Creates an animated contact sheet of avg. morphed cells across map points

In [2]:
!pwd
!date

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/MovieALPBaseline
Sun Jun 26 02:10:27 PDT 2022


In [3]:
import os
import sys
import json
import importlib
import concurrent
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

sys.path.insert(1, '../tools')
import common

In [4]:
# Controller form cvapipe_analysis
path_config = Path("/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/")
control = controller.Controller(general.load_config_file(path_config))
print(control.get_staging())

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance


In [5]:
device = io.LocalStagingIO(control)
df = device.load_step_manifest("preprocessing")
space = shapespace.ShapeSpace(control)
space.execute(df)

In [6]:
variables = control.get_variables_values_for_aggregation()
df_agg = shapespace.ShapeSpaceBasic(control).get_aggregated_df(variables)
df_agg = df_agg.drop(columns=["structure"]).drop_duplicates().reset_index(drop=True)
df_agg.head()

,shape_mode,mpId,aggtype,alias
0,NUC_MEM_PC1,1,avg,STR
1,NUC_MEM_PC1,2,avg,STR
2,NUC_MEM_PC1,3,avg,STR
3,NUC_MEM_PC1,4,avg,STR
4,NUC_MEM_PC1,5,avg,STR


In [7]:
shape_modes = control.get_shape_modes()
df_agg = df_agg.loc[df_agg.shape_mode.isin(shape_modes)]
df_agg = df_agg.loc[df_agg.mpId==control.get_center_map_point_index()]

In [8]:
def process_this_gene(gene):
    row_w_gene = row.copy()
    row_w_gene["structure"] = gene
    rep = device.read_agg_parameterized_intensity(row_w_gene)
    morphed = cytoparam.morph_representation_on_shape(
        img=domain,
        param_img_coords=coords_param,
        representation=rep
    )
    return np.stack([domain_nuc, domain_mem, morphed], axis=0)

In [9]:
genes = control.get_gene_names()

In [24]:
importlib.reload(common)

<module 'common' from '/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/MovieALPBaseline/../tools/common.py'>

In [11]:
morphs = dict([(gene, []) for gene in genes])
for index, row in tqdm(df_agg.iterrows(), total=len(df_agg)):
    domain, domain_nuc, domain_mem, coords_param = common.get_map_point_shape(control, device, row)
    with concurrent.futures.ProcessPoolExecutor(control.get_ncores()) as executor:
        morphed = list(tqdm(executor.map(process_this_gene, genes), total=len(genes), leave=False))
    for gid, gene in enumerate(genes):
        morphs[gene].append(morphed[gid])

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [30]:
pct = {}
for gene in genes:
    val = 90
    pct[gene] = val

In [31]:
%%time
contrast = {}
for gene in genes:
    mode = {"nuc":"center_nuc","mem":"center_nuc","gfp":"center_nuc"}
    contrast[gene] = common.Projector.get_shared_morphed_max_based_on_pct_for_zy_views(
        instances = morphs[gene],
        pct = pct[gene],
        mode = mode,
        func = np.max
    )

CPU times: user 2min 45s, sys: 4.1 s, total: 2min 49s
Wall time: 2min 49s


In [32]:
log = {"contrast": contrast, "version": {}}
for fname in ["../tools/common.py", "MakeContrastTable.ipynb"]:
    with open(fname, "r") as ftxt:
        log["version"][fname] = ftxt.read()
with open("contrast_V3_90OnValids.json", "w") as fj:
    json.dump(log, fj, indent=4)

In [33]:
common.now("complete")

complete 2022-06-26 11:55:33
